In [10]:
import os as os
import sys as sys
import re as re
import pandas as pd
import numpy as np
import json as json
from subprocess import run


pd.options.display.max_columns = 100
pd.options.display.min_rows = None
pd.options.display.max_rows = 20
pd.options.display.max_colwidth = 100

from config import RANDOM_STATE, RESULTS_PATH

# RANDOM_STATE = None

In [11]:
run(f"jupyter nbconvert --to=python --output-dir='./' gene_symbol_sequences.ipynb", shell=True)
run(f"jupyter nbconvert --to=python --output-dir='./' ngram_analysis.ipynb", shell=True)
run(f"jupyter nbconvert --to=python --output-dir='./' classification.ipynb", shell=True)

[NbConvertApp] Converting notebook gene_symbol_sequences.ipynb to python


In [ ]:
# run(f"python gene_symbol_sequences.py", shell=True)
# run(f"python ngram_analysis.py", shell=True)
run(f"python classification.py", shell=True)

In [7]:
# df = pd.read_pickle(RESULTS_PATH.joinpath(f'classification_protein_{RANDOM_STATE}.pkl'))

# df = df.groupby(['classifier']).apply(lambda x: x.loc[x['mean_test_score'] == x['mean_test_score'].max()].iloc[0:1] if x.name != 'DummyClassifier' else x)

# df = df[['classifier', 'params', 'mean_test_score', 'std_test_score']]

# df = df.reset_index(drop=True)

# df

In [8]:
# df = pd.read_pickle(RESULTS_PATH.joinpath(f'classification_exon_{RANDOM_STATE}.pkl'))

# df = df.groupby(['classifier']).apply(lambda x: x.loc[x['mean_test_score'] == x['mean_test_score'].max()].iloc[0:1] if x.name != 'DummyClassifier' else x)

# df = df[['classifier', 'params', 'mean_test_score', 'std_test_score']]

# df = df.reset_index(drop=True)

# df

In [9]:
# df = pd.read_pickle(RESULTS_PATH.joinpath(f'classification_unspliced_{RANDOM_STATE}.pkl'))

# df = df.groupby(['classifier']).apply(lambda x: x.loc[x['mean_test_score'] == x['mean_test_score'].max()].iloc[0:1] if x.name != 'DummyClassifier' else x)

# df = df[['classifier', 'params', 'mean_test_score', 'std_test_score']]

# df = df.reset_index(drop=True)

# df